In [11]:
import torch
torch.cuda.is_available()

True

In [12]:
# from datasets import load_dataset

import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from langchain.schema import Document
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore
from tqdm.auto import tqdm
tqdm.pandas()

import pandas as pd
pd.set_option('display.max_columns', None)

d:\Vscode_projects\Intro_to_LLM\habr_rag_bot\.habr_rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# ds = load_dataset("IlyaGusev/habr", trust_remote_code=True)
# ds.save_to_disk("D:\Vscode_projects\Intro_to_LLM\data\habr_data")
# df = ds["train"].to_pandas()

In [14]:
df10 = pd.read_csv('../data/habr_10_percent.csv')

In [15]:
df10.sample(3)

,id,language,url,title,text_markdown,text_html,author,original_author,original_url,lead_html,lead_markdown,type,time_published,statistics,labels,hubs,flows,tags,reading_time,format,complexity,comments
28550,193378,ru,https://habr.com/ru/post/193378/,Установка и настройка TeamSpeak 3 сервера на VDS,В данной статье речь пойдет об установке серве...,"<div xmlns=""http://www.w3.org/1999/xhtml"">В да...",Dmitry Kolesnikov (mittus),NaN,NaN,В данной статье речь пойдет об <b>установке се...,В данной статье речь пойдет об установке серве...,article,1378909610,"{'commentsCount': 17, 'favoritesCount': 114, '...",['sandbox'],['s_admin'],['admin'],['teamspeak' 'debian' 'ubuntu' 'ts3'],3.0,tutorial,NaN,"{'id': array([6715744, 6715764, 6715776, 67164..."
13904,166575,ru,https://habr.com/ru/post/166575/,Показ масштабируемых изображений,Традиционная работа HTML-страниц с картинками ...,"<div xmlns=""http://www.w3.org/1999/xhtml""><a h...",spmbt (spmbt),NaN,NaN,"<a href=""https://habr.com/images/px.gif#%3D%22...",Традиционная работа HTML-страниц с картинками ...,article,1358839525,"{'commentsCount': 5, 'favoritesCount': 76, 're...",[],['webdev' 'javascript'],['develop'],['просмотр изображений' 'HabrAjax'],10.0,tutorial,NaN,"{'id': array([5753983, 5754433, 5757399, 57575..."
3134,160621,ru,https://habr.com/ru/post/160621/,Какого цвета Марс?,"Сегодня я возьмусь за тему, которая является у...","<div xmlns=""http://www.w3.org/1999/xhtml"">Сего...",Виталий Егоров (Zelenyikot),NaN,NaN,"Сегодня я возьмусь за тему, которая является у...","Сегодня я возьмусь за тему, которая является у...",article,1354711896,"{'commentsCount': 138, 'favoritesCount': 233, ...",[],['robot' 'photo'],['popsci'],['Марс' 'марсоходы' 'curiosity' 'nasa'],9.0,NaN,NaN,"{'id': array([15694707, 15694709, 15694711, 15..."


In [16]:
df10['text_w_title'] = df10['title'] + "\n" +df10['text_markdown']
df10["text_w_title"] = df10["text_w_title"].astype(str)
df10["url"] = df10["url"].astype(str)
df10["title"] = df10["title"].astype(str)

## Создание векторной БД и добавление туда документов

In [17]:
separators=[
        "\n\n",             # Разделение по абзацам
        "\n",               # Перенос строки
        "[code]", "```",    # Кодовые блоки
        "####",             # Заголовки четвертого уровня
        "###",              # Заголовки третьего уровня
        "##",               # Заголовки второго уровня
        "#",                # Заголовки первого уровня
        ".",                # Точки в предложениях
        " "
    ]
def create_doc_list(df10, chunk_size=1024, chunk_overlap=128, separators=separators):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, separators=separators,
        chunk_overlap=chunk_overlap)
    
    documents = []
    for _, row in df10.iterrows():
        chunks = text_splitter.split_text(row["text_w_title"])
        for chunk in chunks:
            documents.append(Document(
                page_content=chunk,
                metadata={"link": str(row["url"]), "title": str(row["title"])}
            ))
    return documents

In [18]:
# docs = create_doc_list(df10)
# embedder = HuggingFaceEmbeddings(model_name="sergeyzh/rubert-tiny-turbo")
# client = QdrantClient(path="../data/langchain_qdrant_10percent")

# client.create_collection(
#     collection_name="embed_collection_10percent",
#     vectors_config=VectorParams(size=312, distance=Distance.COSINE),
# )

# vector_store = QdrantVectorStore(
#     client=client,
#     collection_name="embed_collection_10percent",
#     embedding=embedder,
# )

# vector_store.add_documents(docs)

### Подключение к Qdrant

In [19]:
client = QdrantClient(path="../data/langchain_qdrant_10percent")
embedder = HuggingFaceEmbeddings(model_name="sergeyzh/rubert-tiny-turbo")
collection_name = "embed_collection_10percent"

vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embedder
)

C:\Users\ivann\AppData\Local\Temp\ipykernel_7068\2378406736.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="sergeyzh/rubert-tiny-turbo")


In [20]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":5})

## Тестирование генерации ответов по ретривнуты докуменатам

In [9]:
from langchain.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain import hub
from tqdm import tqdm

model_name = "herenickname/t-tech_T-lite-it-1.0:q4_k_m"

# Инициализация модели Ollama через LangChain
llm = OllamaLLM(model=model_name, base_url= "http://localhost:11434")

'Ранее мы обсуждали гоночные машины. Если у тебя есть какие-то вопросы или ты хочешь продолжить разговор на эту тему, дай знать!'

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

chain = prompt | llm

def generate_answer(query, retriever):
    # Получение релевантных документов
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n ---".join([doc.page_content for doc in relevant_docs])

    # Генерация текста через Ollama
    response = chain.invoke({
        "context" : context,
        "question" : query
    })
    return context, response

# Запросы для генерации
queries = ["Что такое FAISS?", "Как работает GPT?", "Что такое векторные базы данных?"]

for query in tqdm(queries, desc="Processing queries"):
    context, answer = generate_answer(query, retriever)
    print(f"Вопрос: {query}")
    print(f"Документы: {context}")
    print(f"Ответ: {answer}\n")
    print(f"\n"*5)

d:\Vscode_projects\Intro_to_LLM\habr_rag_bot\.habr_rag_env\Lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Processing queries:  33%|███▎      | 1/3 [00:03<00:06,  3.46s/it]

Вопрос: Что такое FAISS?
Документы: Одна из таких производных моделей это FISSA (2020 года), как гибрид из более сложных новых и более старых базовых подходов.
Рекомендательные модели, учитывающие последовательность действий, увеличивают точность за счет учета динамики предпочтений. При этом постоянные и глобальные предпочтения недооцениваются. FISSA — это state-of-the-art модель на последовательностях событий с self-attention механизмом, которая балансирует локальные (динамические) предпочтения пользователя и глобальные.
Для локальных (динамических) предпочтений берется SASRec, как хорошо зарекомендовавший себя метод. Для учета глобальных предпочтений используется location-based (основанный на позиции в пространстве) attention-слой.
Здесь используется implicit feedback: то есть на вход поступает последовательность из L последних объектов с которыми взаимодействовал пользователь. Цель — выдать список рекомендаций, в котором реальный следующий объект взаимодействия будет как можно ближе

Processing queries:  67%|██████▋   | 2/3 [00:09<00:04,  4.85s/it]

Вопрос: Как работает GPT?
Документы: ### Как мы используем GPT
GPT используется в нескольких автоматизированных пайплайнах GitLab CI для получения быстрой обратной связи о работе GitLab. CI-пайплайны эталонных архитектур работают с последним пре-релизом и, как правило, запускаются ежедневно или еженедельно. Сразу после тестирования мы анализируем результаты для поиска проблем. Все последние результаты мы публикуем в GPT wiki согласно нашему Transparency value.
GPT также используется в пайплайне сравнительного тестирования, чтобы увидеть, как изменяется производительность GitLab от релиза к релизу. Эти результаты важны, так как показывают картину улучшения производительности. Результаты сравнения бенчмарков также доступны в GPT wiki.
 ---### Как вы можете использовать GPT
Мы давно решили, что хотим следовать тем же принципам открытого исходного кода, как и для нашего основного продукта, поэтому при создании GPT мы ориентировались на всех пользователей, а не на то, чтобы сделать его искл

Processing queries: 100%|██████████| 3/3 [00:13<00:00,  4.41s/it]

Вопрос: Что такое векторные базы данных?
Документы: . Зачем это в государственнной базе данных?»
 ---В этой статье мы рассмотрим системы управления базами данных, а также узнаем, что же представляют собой реляционные и нереляционные базы и в чём заключаются их особенности.
Ниже мы собрали список
основных пунктов, чтобы вам было проще ориентироваться по статье:
   Программное обеспечение для работы с современными базами данных
   Основные функции программного обеспечения баз данных
   Преимущества и недостатки СУБД
   Реляционные базы данных
   Нереляционные базы данных
   Как выбрать тип базы?
## Программное обеспечение для работы с современными базами данных
 ---\- Структуры данных и тип данных что есть что?
Структура данных \- это контейнер, который хранит данные в определенном макете. Основные структуры данных: массивы, стеки, очереди, связные списки, графы, деревья, префиксные деревья, хэш-таблицы;
Каждая переменная имеет свой тип, это определяет какие операции над ней можно произв